In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import EncoderRNN, AttnDecoderRNN
import json
import helpers


encoder_dict = torch.load('./model_v4.pt', map_location=torch.device('cpu'))['encoder_state_dict']
decoder_dict = torch.load('./model_v4.pt', map_location=torch.device('cpu'))['decoder_state_dict']
    
with open('../project_data/project_train_data_instr.json') as json_file:
    train_data = json.load(json_file)

In [2]:
N_EPOCHS = 15
LEARNING_RATE = 0.01
REPORT_EVERY = 1000
HIDDEN_DIM = 256
#BATCH_SIZE = 20
#N_LAYERS = 1
teacher_forcing_ratio = 1
TRAIN_SET_SIZE = 1000
n_words = 43863
MAX_LENGTH = 70

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(10)

encoder = EncoderRNN(n_words, HIDDEN_DIM).to(device)
decoder = AttnDecoderRNN(HIDDEN_DIM, n_words, max_length=MAX_LENGTH).to(device)

encoder.load_state_dict(encoder_dict)
decoder.load_state_dict(decoder_dict)

<All keys matched successfully>

In [3]:
encoder.eval()
decoder.eval()

recipe_step_pairs, idx2word, word2idx, ml = helpers.get_tensor_data()
n_words = len(word2idx)
print(recipe_step_pairs[0])

Number of short ingredient lists:  130567
Average ingredient list length: 14.175872007959267
No ingredients filtered
Max instruction step length:  70
Number of long instructions:  61032
Average instruction length: 149.95270527301457
Total instruction steps:  489828
Recipes filtered:  61455
Recipes left after filtering:  75241
Recipe step pairs:  223824
(tensor([[43860],
        [   17],
        [   18],
        [   19],
        [   20],
        [   21],
        [   22],
        [   23],
        [   24],
        [   21],
        [   25],
        [   26],
        [   27],
        [   28],
        [   29],
        [   30],
        [   31],
        [   32],
        [   33],
        [   27],
        [43862]]), tensor([[43860],
        [   34],
        [   35],
        [   36],
        [    1],
        [   37],
        [   38],
        [   39],
        [   40],
        [   41],
        [   42],
        [   43],
        [   27],
        [   44],
        [    2],
        [   45],
        [   4

In [162]:
from random import choice
from helpers import idx_to_words
from nltk.tokenize import sent_tokenize, word_tokenize
import re

def evaluate(encoder, decoder, input_tensor, gold_standard):
    with torch.no_grad():
        max_length = MAX_LENGTH
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(device)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        loss = 0
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[word2idx['<SOS>']]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if di < len(gold_standard):
                loss += loss_function(decoder_output, gold_standard[di])
            else:
                loss += loss_function(decoder_output, gold_standard[-1])
            if topi.item() == word2idx['<EOS>']:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx2word[str(topi.item())])

            decoder_input = topi.squeeze().detach()

        return decoded_words, loss.item()/len(gold_standard), decoder_attentions

    
def random_evaluate(evaluation_data, n=10):
    for i in range(n):
        pair = choice(evaluation_data)
        print('Instruction step', idx_to_words(pair[0], idx2word))
        print('Next step', idx_to_words(pair[1], idx2word))
        output_words, loss, attentions = evaluate(encoder, decoder, pair[0].to(device), pair[1].to(device))
        output_sentence = ' '.join(output_words)
        print('Generated instructions', output_sentence)
        print("Loss: ", loss)
        print('')
        
        
def evaluate_with_given_input(instruction_pair):
    output_words, loss, attentions = evaluate(encoder, decoder, instruction_pair[0].to(device), instruction_pair[1].to(device))
    output_sentence = ' '.join(output_words)
    return output_sentence, loss, attentions

    
def tokenize(instruction_step):
    words_tokenized = word_tokenize(instruction_step)
    return words_tokenized


def add_helper_tokens(step_tokenized):
    new_step = ['<SOS>']
    new_step.extend(step_tokenized)
    new_step.append('<EOS>')
    return new_step

def to_idx_repr(tokenized_instruction):
    idx_list = [word2idx[w] if w in word2idx else word2idx['<LN>'] for w in tokenized_instruction]
    instr_tensors = torch.tensor(idx_list).view(-1, 1)
    return instr_tensors
    

def prepare_input_instruction(text):
    tokenized = tokenize(text)
    tokenized_h = add_helper_tokens(tokenized)
    tensor = to_idx_repr(tokenized_h)
    return tensor


def remove_helper_tokens(text):
    helpers_r = r'(<SOS>)|(<EOS>)'
    cleaned_text = re.sub(helpers_r, "", text, count=2)
    return cleaned_text


def preprocess_instruction_data_from_recipes(recipes, limit):
    preprocessed = []
    filtered_out = 0
    for rec in recipes:
        rec_steps = []
        use_rec = True
        for step in rec:
            if len(step) < limit:
                tensor_step = prepare_input_instruction(step)
                rec_steps.append(tensor_step)
            else:
                use_rec = False
                filtered_out = filtered_out + 1
        if use_rec:
            preprocessed.append(rec_steps)
    print(filtered_out, " recipes filtered out")
    return preprocessed



def generate_next_steps(first_step):
    print('Input: ', first_step)
    steps = []
    made_up_instruction = first_step
    i = 1
    while len(steps) < 10 and made_up_instruction != "<SOS> <EOS>":
        tensor = prepare_input_instruction(made_up_instruction)
        made_up_instruction = evaluate_with_given_input(tensor)
        steps.append(made_up_instruction)
        print(i,".", remove_helper_tokens(made_up_instruction))
        i = i + 1
        

def get_instruction_steps(recipes):
    recipe_step_pairs = []
    for recipe in recipes:
        for i, instr_step in enumerate(recipe[:-1]):
            recipe_step_pairs.append((instr_step, recipe[i+1]))
    print("Recipe step pairs: ", len(recipe_step_pairs))
    return recipe_step_pairs




#made_up_instruction = "chicken Italian-seasoned bread crumbs small onion cloves garlic taste oil Mix ground chicken , 1/4 cup bread crumbs , onion , egg , garlic , salt , and black pepper in a bowl . Moisten hands and shape chicken mixture , 2 tablespoons at a time , into flat , oval-shaped patties ."
#generate_next_steps(made_up_instruction)

loss_function = nn.NLLLoss()
random_evaluate(recipe_step_pairs)

Instruction step <SOS> Mix crabmeat , 1/4 cup of the bread crumbs , mayo , egg , barbecue sauce and onion in large bowl . Cover . Refrigerate 2 hours . <EOS>
Next step <SOS> Shape crabmeat mixture into 8 patties . Coat patties with remaining bread crumbs . Spray each patty on both sides with no stick cooking spray . Cook patties in butter in large skillet 4 to 5 minutes on each side or until firm and golden brown on both sides . <EOS>
Generated instructions <SOS> Preheat oven to 400 degrees F ( 200 degrees C ) . <EOS>
Loss:  3.3277708200307994

Instruction step <SOS> Place the noodles , ham , cheese , soup and milk in a 9x9 inch casserole dish and mix well . <EOS>
Next step <SOS> Bake at 375 degrees F ( 190 degrees C ) for 25 to 30 minutes . <EOS>
Generated instructions <SOS> Bake in preheated oven for 30 minutes , or until bubbly and lightly browned on top . <EOS>
Loss:  11.204986572265625

Instruction step <SOS> Heat olive oil and 1 tablespoon butter in large nonstick skillet over me

In [163]:
cookstr = [json.loads(line) for line in open('../../original_data/cookstr-recipes.json', 'r')]

In [164]:
test_recs = [rec['instructions'] for rec in cookstr]
limit = 120
prcessed = preprocess_instruction_data_from_recipes(test_recs, limit)
prcessed = [r for r in prcessed if len(r) > 0]


test_data_steps = get_instruction_steps(prcessed)

26620  recipes filtered out
Recipe step pairs:  469


In [165]:
total_loss = 0

for t in test_data_steps:
    output, loss, attention = evaluate_with_given_input(t)
    total_loss += loss
    
print("Average loss for test set: ", total_loss/len(test_data_steps))

Average loss for test set:  16.6299967519425


In [166]:
random_evaluate(test_data_steps)

Instruction step <SOS> Combine all the ingredients in a bowl and mix well . Pour the mixture into the dish . <EOS>
Next step <SOS> Bake for 50 to 60 minutes , or until set . Serve as soon as possible . <EOS>
Generated instructions <SOS> To serve , spoon the top of the soup in the centre of the plate , pour the mixture into the hot water , and pour in the hot water . <EOS>
Loss:  16.68449642783717

Instruction step <SOS> In a small bowl , whisk together the vinaigrette ingredients . <EOS>
Next step <SOS> Arrange the lettuce on individual salad plates and spoon the dressing over the top just before serving . <EOS>
Generated instructions <SOS> In a large bowl , mix together the flour , salt , and sugar . Stir in the eggs , and mix well . <EOS>
Loss:  12.589152526855468

Instruction step <SOS> On a lightly-floured surface , divide the dough into 4 pieces , and pat into 1-inch-thick rounds . <EOS>
Next step <SOS> Bake on an ungreased baking sheet for 35 minutes , until golden . <EOS>
Genera